
## Этап 1: Сбор данных



### Оптимизированный процесс сбора сырых данных TMDB 

1. **Параллельная загрузка**
   - 20 одновременных потоков
   - Обработка 750 страниц (рассчитанный лимит согласно ограничениям проекта)
   - Автоматический пропуск пустых страниц

2. **Детализация информации**  

   Для каждого фильма получаем 26 колонок со свойствами:  
   > `id` `title` `original_title` `overview` `release_date` `popularity` `imdb_average` `imdb_count` `imdb_id` `director` `runtime` `tagline` `production_countries` `original_language` `production_companies` `genres` `full_cast` `top_cast` `secondary_cast` `episodic_cast`

3. **Контроль качества**
   ```python
   response.raise_for_status()  # проверка HTTP-ошибок
   if not data.get('results'):  # пропуск пустых страниц
   ```

**Защитные механизмы**
- Пауза `0.1 сек` между запросами
- Повторная попытка при сбоях
- Логирование всех ошибок


**Особенности:**
- Автоматическое создание папки `data/raw`
- Сохранение в CSV с кодировкой UTF-8
- Вывод базовой статистики после завершения


> **Производительность**: На тестах сбор 10,000 записей занимает ~5 минут при стабильном интернет-соединении.


In [4]:
import requests
import pandas as pd
import time
import os
from concurrent.futures import ThreadPoolExecutor, as_completed


- узнав id жанра Sci-Fi в документации TMDB, собираем данные о научно-фантастических фильмах с TMDB
- `total_pages:` устанавливаем, сколько страниц страниц парсить (на каждой до 20 фильмов)

`fetch_movies_page` - функция для получения данных о фильмах с одной страницы API TMDB.

- делает GET-запрос к TMDB API для получения списка фильмов по жанру
- обрабатывает возможные ошибки запроса
  - проверяет HTTP статус через `raise_for_status()`
  - ловит исключения и логирует ошибки
  - возвращает пустой DataFrame при проблемах 
- конвертирует JSON-ответ в pandas DataFrame


`get_movie_details` -- функция дополняет базовую информацию о фильмах новыми колонками. 

**роли функции:**
- **принимает** на вход dataframe с минимальными данными о фильмах (id, название и т.д.)
- **возвращает** новый dataframe с расширенной информацией

**что делает внутри:**
1. для каждого фильма в цикле:
   - строит url запроса к api, включая:
     - id фильма
     - api-ключ
     - дополнительные данные (credits — актёры, external_ids — imdb)
   - обрабатывает возможные ошибки запроса
2. на базе каждой строки присваивает значения в новых колонках:

| **Колонка**                | **Тип данных** | **Описание**                                                                 | **Источник**                     |
|----------------------------|----------------|------------------------------------------------------------------------------|----------------------------------|
| **id**                     | int            | Уникальный ID фильма в TMDB                                                   | Базовый запрос                   |
| **title**                  | str            | Локализованное название                                                       | Базовый запрос                   |
| **original_title**         | str            | Оригинальное название                                                         | Базовый запрос                   |
| **overview**               | str            | Описание сюжета                                                               | Базовый запрос                   |
| **release_date**           | str            | Дата выхода (YYYY-MM-DD)                                                      | Базовый запрос                   |
| **popularity**             | float          | Индекс популярности TMDB                                                      | Базовый запрос                   |
| **imdb_average**           | float          | Средний рейтинг IMDb (0-10)                                                   | `vote_average` из базовых        |
| **imdb_count**             | int            | Количество оценок IMDb                                                        | `vote_count` из базовых          |
| **imdb_count**             | int            | Количество оценок IMDb                                                        | `external_ids`         |
| **director**               | list[str]      | Режиссёр фильма                                                              | `credits.crew` (job == 'Director')|
| **producer**               | list[str]      | Продюсеры фильма                                                              | `credits.crew` (job == 'Producer')|
| **writer**                 | list[str]      | Сценаристы фильма                                                             | `credits.crew` (job == 'Writer')|
| **editor**                 | list[str]      | Монтажеры фильма                                                              | `credits.crew` (job == 'Editor') |
| **cinematographer**        | list[str]      | Операторы (фотооператоры)                                                     | `credits.crew` (job == 'Director of Photography') |
| **budget**                 | int            | Бюджет фильма ($)                                                             | Детальный запрос                 |
| **revenue**                | int            | Сборы фильма ($)                                                              | Детальный запрос                 |
| **runtime**                | int            | Длительность фильма в минутах                                                 | Детальный запрос                 |
| **tagline**                | str            | Слоган фильма                                                                 | Детальный запрос                 |
| **production_countries**   | list[str]      | Страны производства (например ["US", "UK"])                                    | Детальный запрос                 |
| **original_language**      | str            | Язык оригинала (код ISO 639-1)                                                | Детальный запрос                 |
| **production_companies**   | list[str]      | Компании-производители                                                        | Детальный запрос                 |
| **imdb_id**                | str            | ID фильма на IMDb (например "tt1234567")                                      | `external_ids`                   |
| **genres**                 | list[str]      | Жанры фильма (например ["Sci-Fi", "Adventure"])                               | Детальный запрос                 |
| **full_cast**              | list[str]      | Весь актёрский состав                                                         | `credits.cast`                   |
| **top_cast**               | list[str]      | 5 главных актёров                                                            | `credits.cast[:5]`               |
| **secondary_cast**         | list[str]      | Актеры на второстепенных ролях (5-15 в списке)                                | `credits.cast[5:15]`             |
| **episodic_cast**          | list[str]      | Эпизодические роли (актёры после 15-го места)                                 | `credits.cast[15:]`              |


1. **Детальные запросы**:
   - самые простые для вызова поля, такие как `budget`, `revenue`, `runtime`, `tagline`, `genres` и другие, извлекаются из детализированных запросов
2. **`director`**, **`producer`**, **`writer`**, **`editor`**, **`cinematographer`**:
   - эти колонки извлекаются из поля `crew` в `credits`, где каждый человек имеет роль, указанную в поле `job`.
   - в каждой из этих колонок будут храниться имена людей, занимающихся соответствующими ролями.

3. **`full_cast`**, **`top_cast`**, **`secondary_cast`**, **`episodic_cast`**:
   - Эти данные извлекаются из поля `cast` в `credits` и представляют собой список актеров. В `full_cast` будет весь актерский состав, в `top_cast` — первые 5 актеров, в `secondary_cast` — актеры с 6 по 15, а в `episodic_cast` — актеры с 16-го места и дальше.

4. **`production_companies`**, **`production_countries`**:
   - Эти данные извлекаются из поля `production_companies` и `production_countries` в детальном запросе фильма. Они содержат названия компаний и стран, участвующих в производстве фильма.


In [5]:
api_key = 'bcd3dc372c98663f3b351a83592efe6a'

def get_sci_fi_movies(total_pages=750): 
    # id жанра научной фантастики в TMDB
    sci_fi_genre_id = 878

    # здесь будем хранить все собранные данные
    all_movies = pd.DataFrame()
    
    # используем многопоточность для ускорения
    with ThreadPoolExecutor(max_workers=20) as executor:
        # создаем задачи для каждой страницы
        tasks = {
            executor.submit(fetch_movies_page, page, sci_fi_genre_id): page 
            for page in range(1, total_pages + 1)
        }
        
        # обрабатываем результаты по мере готовности
        for future in as_completed(tasks):
            page_data = future.result()
            if not page_data.empty:
                all_movies = pd.concat([all_movies, page_data], ignore_index=True)
            
            # небольшая пауза, чтобы не нагружать API
            time.sleep(0.1)
    
    return all_movies

def fetch_movies_page(page, genre_id):
    url = f'https://api.themoviedb.org/3/discover/movie?api_key={api_key}&with_genres={genre_id}&page={page}'
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # проверяем на ошибки HTTP
        data = response.json()
        
        # если на странице нет фильмов, возвращаем пустой датафрейм
        if not data.get('results'):
            return pd.DataFrame()
        
        page_df = pd.DataFrame(data['results'])
        return get_movie_details(page_df)
    
    except Exception as e:
        print(f"проблема со страницей {page}: {e}")
        return pd.DataFrame()

def get_movie_details(movies_df):
    detailed_movies = []
    
    for _, movie in movies_df.iterrows():
        try:
            # запрашиваем детали для каждого фильма
            detail_url = f'https://api.themoviedb.org/3/movie/{movie["id"]}?api_key={api_key}&append_to_response=credits,external_ids'
            detail_response = requests.get(detail_url)
            detail_response.raise_for_status()
            details = detail_response.json()
            
            # формируем полную запись о фильме
            movie_data = {
                'id': movie['id'],
                'title': movie['title'],
                'original_title': movie['original_title'],
                'overview': movie['overview'],
                'release_date': movie['release_date'],
                'popularity': movie['popularity'],
                'imdb_average': movie['vote_average'], # рейтинг imdb для фильма или сериала
                'imdb_count': movie['vote_count'], # количество голосов на imdb
                'imdb_id': details.get('external_ids', {}).get('imdb_id', None),

                # фильтрация и извлечение ролей съемочной команды из списка объектов
                'director': [ person['name'] for person in details.get('credits', {}).get('crew', []) if person['job'] == 'Director'], # режиссер
                'producer': [person['name'] for person in details.get('credits', {}).get('crew', []) if person['job'] == 'Producer'], # продюссер
                'writer': [person['name'] for person in details.get('credits', {}).get('crew', []) if person['job'] == 'Writer'], # сценарист
                'editor': [person['name'] for person in details.get('credits', {}).get('crew', []) if person['job'] == 'Editor'], # режиссер монтажа
                'cinematographer': [person['name'] for person in details.get('credits', {}).get('crew', []) if person['job'] == 'Director of Photography'], # оператор
                
                # финансовая информация
                'budget': details.get('budget', 0),
                'revenue': details.get('revenue', 0),
                
                # технические детали
                'runtime': details.get('runtime', 0),
                'tagline': details.get('tagline', ''),
                'production_countries': [country['name'] for country in details.get('production_countries', [])], # формируем список из стран продакшена
                'original_language': details.get('original_language', ''),
                'production_companies': [company['name'] for company in details.get('production_companies', [])], # формируем список из названий производящих компаний
                
                # внешние ссылки
                'imdb_id': details.get('external_ids', {}).get('imdb_id', ''),
                
                # жанры
                'genres': [genre['name'] for genre in details.get('genres', [])],
                
                # через питоновские срезы составляю несколько колонок про каст фильма
                'full_cast': [
                    actor['name'] for actor in details.get('credits', {}).get('cast', [])[:]
                ],
                'top_cast': [
                    actor['name'] for actor in details.get('credits', {}).get('cast', [])[:5] # главные герои
                ],
                'secondary_cast': [
                    actor['name'] for actor in details.get('credits', {}).get('cast', [])[5:15] # второстепенные герои
                ],
                'episodic_cast': [
                    actor['name'] for actor in details.get('credits', {}).get('cast', [])[15:] # эпизодические герои
                ]
            }
            
            detailed_movies.append(movie_data)
            
        except Exception as e:
            print(f"не удалось получить данные для фильма {movie['id']}: {e}")
    
    return pd.DataFrame(detailed_movies)

# основной блок выполнения
if __name__ == '__main__':
    # собираем данные
    print("идет сбор данных...")
    sci_fi_movies = get_sci_fi_movies()
    
    # сохраняем результат
    os.makedirs('data/raw', exist_ok=True)
    sci_fi_movies.to_csv('data/raw/sci_fi_movies_tmdb.csv', index=False)


идет сбор данных...
проблема со страницей 501: 400 Client Error: Bad Request for url: https://api.themoviedb.org/3/discover/movie?api_key=bcd3dc372c98663f3b351a83592efe6a&with_genres=878&page=501
проблема со страницей 502: 400 Client Error: Bad Request for url: https://api.themoviedb.org/3/discover/movie?api_key=bcd3dc372c98663f3b351a83592efe6a&with_genres=878&page=502
проблема со страницей 504: 400 Client Error: Bad Request for url: https://api.themoviedb.org/3/discover/movie?api_key=bcd3dc372c98663f3b351a83592efe6a&with_genres=878&page=504
проблема со страницей 503: 400 Client Error: Bad Request for url: https://api.themoviedb.org/3/discover/movie?api_key=bcd3dc372c98663f3b351a83592efe6a&with_genres=878&page=503
проблема со страницей 505: 400 Client Error: Bad Request for url: https://api.themoviedb.org/3/discover/movie?api_key=bcd3dc372c98663f3b351a83592efe6a&with_genres=878&page=505
проблема со страницей 506: 400 Client Error: Bad Request for url: https://api.themoviedb.org/3/disco

In [65]:
# выводим статистику и осматриваем датасет на целостность
print(f"собрано фильмов: {len(sci_fi_movies)}")
df = pd.read_csv('data/raw/sci_fi_movies_tmdb.csv')

display(df.sort_values('release_date', ascending=True))

собрано фильмов: 10000


,id,title,original_title,overview,release_date,popularity,imdb_average,imdb_count,imdb_id,director,...,runtime,tagline,production_countries,original_language,production_companies,genres,full_cast,top_cast,secondary_cast,episodic_cast
9796,189571,The Mechanical Butcher,La Charcuterie mécanique,A butcher puts a full-grown live pig into his ...,1896-04-18,0.398500,5.7,45,tt0212052,['Louis Lumière'],...,1,NaN,['France'],fr,['Lumière'],"['Comedy', 'Science Fiction']",[],[],[],[]
7514,119974,Turn-of-the-Century Surgery,Chirurgie fin de siècle,George Mélies made a version of this a few yea...,1900-01-01,0.068990,4.9,21,tt0000286,['Alice Guy-Blaché'],...,2,NaN,['France'],fr,['Gaumont'],"['Science Fiction', 'Horror', 'Comedy']",[],[],[],[]
8288,183601,An Over-Incubated Baby,An Over-Incubated Baby,An up to date idea and a great picture. The pr...,1901-08-01,0.077464,4.9,11,tt0212411,['Walter R. Booth'],...,1,NaN,['United Kingdom'],en,['Robert W. Paul'],"['Science Fiction', 'Comedy']",[],[],[],[]
1289,775,A Trip to the Moon,Le Voyage dans la Lune,Professor Barbenfouillis and five of his colle...,1902-06-15,1.042573,7.9,1821,tt0000417,['Georges Méliès'],...,15,NaN,['France'],fr,['Star Film'],"['Adventure', 'Science Fiction']","['Georges Méliès', 'Bleuette Bernon', 'Françoi...","['Georges Méliès', 'Bleuette Bernon', 'Françoi...","['Brunnet', 'Depierre', 'Farjaut', 'Kelm', ""Je...",[]
5512,2963,The Impossible Voyage,Le Voyage à travers l'impossible,"Using every known means of transportation, sev...",1904-11-10,1.014500,7.2,190,tt0000499,['Georges Méliès'],...,20,NaN,['France'],fr,"['Georges Méliès', 'Star Film']","['Adventure', 'Comedy', 'Fantasy', 'Science Fi...","['Georges Méliès', 'Fernande Albeny', ""Jehanne...","['Georges Méliès', 'Fernande Albeny', ""Jehanne...",[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9964,229718,XJ-1,XJ-1,A young man meets a mysterious cyberspace pers...,NaN,0.059824,0.0,0,NaN,['Marius Swanepoel'],...,97,NaN,['South Africa'],en,['Eternal Films'],['Science Fiction'],[],[],[],[]
9974,1384339,Untitled Simon Kinberg Star Wars Film 2,Untitled Simon Kinberg Star Wars Film 2,The second film in a new trilogy of Star Wars ...,NaN,0.053607,0.0,0,NaN,[],...,0,NaN,['United States of America'],en,['Lucasfilm Ltd.'],['Science Fiction'],[],[],[],[]
9975,1381202,The Embers and the Stars,The Embers and the Stars,"An astrophysicist, after making an extraterres...",NaN,0.053607,0.0,0,tt33372918,['William Woods'],...,0,NaN,['Canada'],en,"['Sara Fost Pictures', 'Obvious Allegory', 'Ti...","['Science Fiction', 'Thriller']","['Constance Wu', ""Mark O'Brien"", 'William Fich...","['Constance Wu', ""Mark O'Brien"", 'William Fich...",[],[]
9978,1318314,Irandu Vaanam,இரண்டு வானம்,Upcoming Romantic Fantasy Film Starring Vishnu...,NaN,0.053607,0.0,0,NaN,['Ram Kumar'],...,0,NaN,['India'],ta,['Sathya Jyothi Films'],"['Romance', 'Fantasy', 'Science Fiction']","['Vishnu Vishal', 'Mamitha Baiju']","['Vishnu Vishal', 'Mamitha Baiju']",[],[]


## Этап 2: Обогащение данных

Насмотря на то, что датасет получился разнообразным, в ходе исследования важно подробно изучить не только технические моменты производства фильмов, но и общественные аспекты восприятие научной фантастики. 

Для полной репрезентативности я хочу расширить датасет засчет другого API,  чтобы получить информацию о фестивальных наградах и рейтинге на дргуих рейтингов -  Rotten Tomatoes, Metacritic и Metascore.


### Обогащение датасета колонками из дополнительного API (OMDB)

Для того чтобы проверить, как выглядит разметка данных базы OMDB, мы можем взять случайный фильм из нашего текущего датасета и запросить его полную информацию через API OMDB. Так мы узнаем, какими дополнительными колонками мы сожем обогатить датасет.

In [68]:
OMDB_API_KEY = 'f1f15728'
imdb_id = 'tt0083658'  # 

url = f"http://www.omdbapi.com/?i={imdb_id}&apikey={OMDB_API_KEY}&tomatoes=true&plot=short"
response = requests.get(url)
print(response.json())

{'Title': 'Blade Runner', 'Year': '1982', 'Rated': 'R', 'Released': '25 Jun 1982', 'Runtime': '117 min', 'Genre': 'Action, Drama, Sci-Fi', 'Director': 'Ridley Scott', 'Writer': 'Hampton Fancher, David Webb Peoples, Philip K. Dick', 'Actors': 'Harrison Ford, Rutger Hauer, Sean Young', 'Plot': 'A blade runner must pursue and terminate four replicants who stole a ship in space and have returned to Earth to find their creator.', 'Language': 'English, German, Cantonese, Japanese, Hungarian, Arabic, Korean', 'Country': 'United States, United Kingdom', 'Awards': 'Nominated for 2 Oscars. 13 wins & 22 nominations total', 'Poster': 'https://m.media-amazon.com/images/M/MV5BOWQ4YTBmNTQtMDYxMC00NGFjLTkwOGQtNzdhNmY1Nzc1MzUxXkEyXkFqcGc@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.1/10'}, {'Source': 'Rotten Tomatoes', 'Value': '89%'}, {'Source': 'Metacritic', 'Value': '84/100'}], 'Metascore': '84', 'imdbRating': '8.1', 'imdbVotes': '842,743', 'imdbID': 'tt0083658', 'T

#### Особенннсти сбора данных
1. **Параллельная загрузка**  
   - 5 одновременных потоков  
   - Обработка данных для каждого фильма по IMDb ID  

2. **Детализация информации**  

   Для каждого фильма извлекаем данные о наградах и рейтингах с платформ:
   > `Awards` (Награды), `Rotten_Tomatoes` (Рейтинг на Rotten Tomatoes), `Metacritic` (Рейтинг на Metacritic), `Metascore` (Общий рейтинг)

3. **Контроль качества**  
   Проверка успешности запроса и обработка ошибок:  
   ```python
   response.raise_for_status()  # проверка HTTP-ошибок
   if not movie_details:  # пропуск фильмов без данных
   ```

**Защитные механизмы**
- Пауза от `0.5 до 2 секунд` между запросами для предотвращения перегрузки сервера и снижения нагрузки на API
- Повторная попытка при сбоях
- Логирование ошибок для отслеживания проблемных запросов

**Особенности:**
- Автоматическое добавление данных о наградах и рейтингах в основной датафрейм
- Сохранение дополнительных данных в том же CSV
- Обработка данных в реальном времени, с логированием хода работы

> **Производительность**: На тестах сбор дополнительных данных для 10,000 фильмов занимает ~10-15 минут при стабильном интернет-соединении.

---

#### Пояснение к коду:

- **`extract_movie_data(movie_details)`** — функция извлекает нужные данные о фильме, такие как награды и рейтинги с различных платформ.

- **`response.status_code == 200`** — проверка на успешный HTTP-статус:
    - В случае ошибки возвращает `None`
    - При успешном запросе преобразует ответ в Python-словарь с помощью `response.json()`

- **Как реализуется <font color="#f79646">параллельная обработка</font>**:
    - Используется многозадачность с `ThreadPoolExecutor` для обработки фильмов.
    - Ограничение до 5 одновременных потоков.
        ```python
        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = {executor.submit(process_movie, imdb_id): imdb_id for imdb_id in imdb_ids}
        ```
    - Применяется случайная задержка между запросами для снижения нагрузки на API (использование time и random)
      


> **Производительность**: частота выдачи запросов OMDB  десятикратно медленнее, чем TMDB - на объединение 10,000 строк с имеющейся базой <font color="#ff0000">потребовалось почти 3 часа</font> (176 минут) 😭.

In [70]:
import random

def extract_movie_data(movie_details):
    if not movie_details or movie_details.get('Response') == 'False': #если сли данных о фильме нет или запрос не удался, возвращаем дефолтные значения
        return {
            'Awards': 'No data',
            'Rotten_Tomatoes': 'N/A',
            'Metacritic': 'N/A',
            'Metascore': 'N/A'
        }
    
    # ищем рейтинг rotten tomatoes в рамках поля 'Ratings', о котором мы узнали, когда изучали разметку
    rt_rating = next((rating['Value'] for rating in movie_details.get('Ratings', []) 
                      if rating['Source'] == 'Rotten Tomatoes'), 'N/A')
    
    return {
        'Awards': movie_details.get('Awards', 'No awards data'),
        'Rotten_Tomatoes': rt_rating,
        'Metacritic': rt_rating.rstrip('%') if rt_rating != 'N/A' else 'N/A',
        'Metascore': 'N/A'
    }

def safe_api_request(imdb_id): ## устанавливаем, как будет работать наш запрос с сервером omdb
    url = f"http://www.omdbapi.com/?i={imdb_id}&apikey={OMDB_API_KEY}&tomatoes=true&plot=short"
    try:
        response = requests.get(url, timeout=15) # ограничение 15 секунд на запрос
        return response.json() if response.status_code == 200 else None
    except Exception:
        return None

def process_movie(imdb_id):
    time.sleep(random.uniform(0.5, 2))
    
    movie_details = safe_api_request(imdb_id)
    movie_data = extract_movie_data(movie_details)
    movie_data['imdb_id'] = imdb_id
    
    return movie_data

# функция для объединения данных иOMDB с основным датасетом
def merge_omdb_data(df): 
    imdb_ids = df['imdb_id'].tolist() # по свойству 'imdb_id' извлекаем строки из основного датасета
    
    processed_data = [] # список записи для выполненных и обрботанных записей
    
    # создаем пул потоков для параллельной обработки
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(process_movie, imdb_id): imdb_id for imdb_id in imdb_ids} # для каждого imdb_id создаем задачу на выполнение
        
        for i, future in enumerate(as_completed(futures), 1):
            result = future.result()
            processed_data.append(result)
            
             # система оповещения и контроля: раз в 50 итераций (обработанных фильмов) счетчик будет активироваться и выводить сообщение
            if i % 50 == 0:
                print(f"добавлено {i} фильмов...")
    
    omdb_df = pd.DataFrame(processed_data) # преобразуем обработанные данные в датафрейм
    df_merged = df.merge(omdb_df, on='imdb_id', how='left') # объединяем исходный датасет с новым датасетом из OMDB, ставя коновые колоки слева
    
    return df_merged

# загружаем сущсетвуюзщйи файл с датасетов
df = pd.read_csv('data/raw/sci_fi_movies_tmdb.csv')

# обхединяем  
df_merged = merge_omdb_data(df)

# сохраняем
df_merged.to_csv('data/raw/sci_fi_movies_merged.csv', index=False)

добавлено 50 фильмов...
добавлено 100 фильмов...
добавлено 150 фильмов...
добавлено 200 фильмов...
добавлено 250 фильмов...
добавлено 300 фильмов...
добавлено 350 фильмов...
добавлено 400 фильмов...
добавлено 450 фильмов...
добавлено 500 фильмов...
добавлено 550 фильмов...
добавлено 600 фильмов...
добавлено 650 фильмов...
добавлено 700 фильмов...
добавлено 750 фильмов...
добавлено 800 фильмов...
добавлено 850 фильмов...
добавлено 900 фильмов...
добавлено 950 фильмов...
добавлено 1000 фильмов...
добавлено 1050 фильмов...
добавлено 1100 фильмов...
добавлено 1150 фильмов...
добавлено 1200 фильмов...
добавлено 1250 фильмов...
добавлено 1300 фильмов...
добавлено 1350 фильмов...
добавлено 1400 фильмов...
добавлено 1450 фильмов...
добавлено 1500 фильмов...
добавлено 1550 фильмов...
добавлено 1600 фильмов...
добавлено 1650 фильмов...
добавлено 1700 фильмов...
добавлено 1750 фильмов...
добавлено 1800 фильмов...
добавлено 1850 фильмов...
добавлено 1900 фильмов...
добавлено 1950 фильмов...
добавл

In [73]:
df = df.drop_duplicates()  # удалим строчки-дубликаты, прежде чем сохранять датафрейм
df = pd.read_csv('data/raw/sci_fi_movies_merged.csv')

df.head()

,id,title,original_title,overview,release_date,popularity,imdb_average,imdb_count,imdb_id,director,...,production_companies,genres,full_cast,top_cast,secondary_cast,episodic_cast,Awards,Rotten_Tomatoes,Metacritic,Metascore
0,2756,The Abyss,The Abyss,A civilian oil rig crew is recruited to conduc...,1989-08-09,3.665648,7.400,3082,tt0096754,['James Cameron'],...,"['20th Century Fox', 'Pacific Western']","['Adventure', 'Action', 'Thriller', 'Science F...","['Ed Harris', 'Mary Elizabeth Mastrantonio', '...","['Ed Harris', 'Mary Elizabeth Mastrantonio', '...","['John Bedford Lloyd', 'Kimberly Scott', 'Chri...","['J. Kenneth Campbell', 'Peter Ratray', 'Micha...",Won 1 Oscar. 9 wins & 16 nominations total,89%,89.0,NaN
1,185,A Clockwork Orange,A Clockwork Orange,"In a near-future Britain, young Alexander DeLa...",1971-12-19,4.283898,8.200,13078,tt0066921,['Stanley Kubrick'],...,"['Warner Bros. Pictures', 'Hawk Films', 'Stanl...","['Science Fiction', 'Crime']","['Malcolm McDowell', 'Patrick Magee', 'Carl Du...","['Malcolm McDowell', 'Patrick Magee', 'Carl Du...","['James Marcus', 'Michael Tarn', 'Miriam Karli...","['Michael Gover', 'Godfrey Quigley', 'Madge Ry...",Nominated for 4 Oscars. 12 wins & 26 nominatio...,86%,86.0,NaN
2,720321,Breathe,Breathe,"Air-supply is scarce in the near future, forci...",2024-04-04,3.567976,5.806,196,tt11540468,['Stefon Bristol'],...,"['Thunder Road', 'Capstone Studios', 'Streamli...","['Action', 'Science Fiction', 'Mystery', 'Thri...","['Jennifer Hudson', 'Milla Jovovich', 'Quvenzh...","['Jennifer Hudson', 'Milla Jovovich', 'Quvenzh...","['Raúl Castillo', 'James Saito', 'Dan Martin',...",[],NaN,14%,14.0,NaN
3,871,Planet of the Apes,Planet of the Apes,Astronaut Taylor crash lands on a distant plan...,1968-02-07,4.094930,7.600,3630,tt0063442,['Franklin J. Schaffner'],...,"['APJAC Productions', '20th Century Fox']","['Science Fiction', 'Adventure', 'Drama', 'Act...","['Charlton Heston', 'Roddy McDowall', 'Kim Hun...","['Charlton Heston', 'Roddy McDowall', 'Kim Hun...","['James Daly', 'Linda Harrison', 'Robert Gunne...","['Martin Abrahams', 'Army Archerd', 'James Bac...",Nominated for 2 Oscars. 6 wins & 5 nominations...,86%,86.0,NaN
4,760873,The Colony,Tides,In the not-too-distant future: after a global ...,2021-08-26,3.646416,5.700,290,tt6506264,['Tim Fehlbaum'],...,"['BerghausWöbke Filmproduktion', 'Vega Film', ...","['Science Fiction', 'Thriller']","['Nora Arnezeder', 'Iain Glen', 'Sarah-Sofie B...","['Nora Arnezeder', 'Iain Glen', 'Sarah-Sofie B...","['Joel Basman', 'Kotti Yun', 'Bella Bading', '...",[],10 wins & 4 nominations,54%,54.0,NaN


### Обогащение данных через выявление дополнительных колонок

 В рамках текущего этапа важно сформировать несколько дополнительных колонок, которые позволят более детально оценить финансовую эффективность, творческую составляющую и временные тренды фильмов.



1) `ROI (Return on Investment)`

    ✅ Показывает отношение прибыли к бюджету, что является важным индикатором <u>финансовой эффективности</u> и <u>рентабельности фильма</u>. ROI позволяет лучше понять, насколько прибыльным был фильм в сравнении с его затратами. 

    🤔 Рассчитывается как (сборы - бюджет) / бюджет.

In [210]:
df_merged['roi'] = (df_merged['revenue'] - df_merged['budget']) / df_merged['budget']

2) `Profit`

    ✅ Эта колонка помогает понять, сколько прибыли фильм принес в сравнении с его бюджетом, показывая реальный финансовый успех.

    🤔 Формула: сборы - бюджет

In [211]:
df_merged['profit'] = df_merged['revenue'] - df_merged['budget']

**3) раскладываем str `Awards` на маленькие колонки**

Изучив датасет, я заметила, что в строковой колонке **Awards** есть множество недостатков, которые затрудняют дальнейшую работу с данными. Во-первых, в колонке смешаны и номинации, и награды, что усложняет их отдельное использование. Во-вторых, многие строки не содержат информации о самих наградах, а лишь упоминают их количество, например "5 nominations" или "3 wins". 

Эта колонка имеет большой потенциал для анализа, ведь она дает нам ключевую информацию о признании фильмов. Однако как с ней работать? Конечно, можно попробовать использовать методы фильтрации, такие как `str.contains()`, для поиска и выделения наград, но этого недостаточно для более глубокого анализа. 

Мы не можем просто извлечь значение "Oscar" или "Golden Globe" и считать их результатом без более точной обработки.

Поэтому я решила подходить к задаче с трех сторон:
1. **Полученные награды** — точно определенные награды, которые фильм получил.
2. **Номинации** — количество номинаций, которые фильм получил.
3. **Отсутствие награды** — случаи, когда фильм не получил награды, но все же упоминается в колонке "Awards".

Далее, мне нужно решить, как лучше работать с этой информацией: вводить *бинарные колонки* (например, есть номинация или нет), *числовые колонки* (количество выигранных наград или номинаций), или, возможно, *категориальные данные* (тип награды).

В качестве первого шага, я решила воспользоваться более надежной стратегией и создать **номинальные колонки** из наград, которые фильм точно получил. Для этого я  сформулировала **гарантированные лингвистические конструкции**. Например, фразы вроде "Won 1 Oscar" или "Won 4 Golden Globes" однозначно указывают на то, что фильм получил эту награду. Такие строки можно выделить и поместить в отдельные категории.

Затем, на основе этих данных, я решила **типировать** награды, разбив их на всевозможные комбинации типов наград, которые упоминаются в базе. Например, если в одной строке указано "Won 1 Oscar" и "Nominated for 2 Golden Globes", это будет означать, что фильм выиграл один "Oscar" и был номинирован дважды на "Golden Globe".

Таким образом, следующий этап работы с этой колонкой заключается в:
1. Выделении всех **конкретных наград**, которые были получены (например, Oscar, Golden Globe).
2. Разделении информации на **независимые колонки** с указанием количества выигранных наград и номинаций.
3. Формировании **бинарных переменных** для определения наличия тех или иных наград (например, есть ли у фильма "Oscar" или "Golden Globe").



Итого получаем следующие новые колонки:

- `wins` — извлекаем количество выигранных наград с помощью регулярных выражений, ищем фразу "Won X" в строках и считаем количество таких случаев.

- `nominations` — аналогично извлекаем количество номинаций, ищем "Nominated for X".

- `awards_type` — определяем, какой тип награды присутствует в строке (Oscar, Golden Globe, BAFTA, Cannes), или присваиваем "Other", если тип не найден.


In [592]:
import re

# Функция для извлечения данных о наградах
def extract_awards_data(awards_string):
    if pd.isna(awards_string) or awards_string == 'No data':
        return {'wins': 0, 'nominations': 0, 'awards_type': 'No data'}
    
    # Ищем количество выигранных наград (например "Won 1", "Won 4")
    wins = sum([int(match) for match in re.findall(r'Won (\d+)', awards_string)])
    
    # Ищем количество номинаций (например "Nominated for 4", "Nominated for 2")
    nominations = sum([int(match) for match in re.findall(r'Nominated for (\d+)', awards_string)])
    
    # Определяем тип награды (Oscar, Golden Globe, BAFTA, Cannes)
    awards_type = 'Other'
    
    # Ищем наличие Oscar, Golden Globe, BAFTA, Cannes в строке
    if 'Oscar' in awards_string:
        awards_type = 'Oscar'
    elif 'Golden Globe' in awards_string:
        awards_type = 'Golden Globe'
    elif 'BAFTA' in awards_string:
        awards_type = 'BAFTA'
    elif 'Cannes' in awards_string:
        awards_type = 'Cannes'

    return {'wins': wins, 'nominations': nominations, 'awards_type': awards_type}

# Применяем функцию ко всем значениям в колонке Awards
df_merged[['wins', 'nominations', 'awards_type']] = df_merged['Awards'].apply(lambda x: pd.Series(extract_awards_data(x)))

# Проверяем результат
df_merged[['title', 'Awards', 'wins', 'nominations', 'awards_type']].head()
df_merged.to_csv('data/raw/sci_fi_movies_merged.csv')


💡 Инсайт: для последующего улучшения базы можно вводить бинарные колонки для каждой из кинонаград (has_oscar, has_golden_globe, has_bafta, has_cannes).

# **Этап 3:** Предобработка и очистка данных

Цель этого этапа — подготовить собранный датасет к анализу: убрать лишние шумы, привести форматы к единому виду и заложить основу для визуализаций, агрегирования и построения моделей.

---





In [593]:
df_merged.info()  #проводим осмотр таблицы

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485858 entries, 0 to 485857
Data columns (total 33 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    485858 non-null  int64  
 1   title                 485858 non-null  object 
 2   original_title        485858 non-null  object 
 3   overview              422264 non-null  object 
 4   release_date          426331 non-null  object 
 5   popularity            485858 non-null  float64
 6   imdb_average          485858 non-null  float64
 7   imdb_count            485858 non-null  int64  
 8   imdb_id               9758 non-null    object 
 9   director              485858 non-null  object 
 10  producer              485858 non-null  object 
 11  writer                485858 non-null  object 
 12  editor                485858 non-null  object 
 13  cinematographer       485858 non-null  object 
 14  budget                485858 non-null  int64  
 15  

### **1. Работа с пропущенными значениями**
- Столбцы `release_date`, `overview`, `tagline`, `roi`, `imdb_id` содержат пропуски.
- Решение:
  - Удалить строки с критически важными пропусками (например, если нет ни даты, ни рейтинга, ни описания).

### **2. Приведение форматов к единому виду**
- Преобразовать `release_date` в формат datetime
- Создать столбец `release_year`
   - ✅ это будет важная колонка для временного анализа, которая позволит изучать влияние временных факторов на популярность, бюджеты и жанры фильмов. `realease_date` не позволяет этого делать.
- Оставить только фильмы, выпущенные до 1 января 2025 года
- Столбцы `Rotten_Tomatoes`, `Metacritic`, `Metascore` — привести к числовому виду, удалив `%`, `N/A` и пр.

### **3. Очистка текстовых полей**
- Удалить лишние пробелы, спецсимволы и единообразно привести регистр в `overview`, `title`, `director`, `genres`, `production_companies` и т.д.
- Преобразовать строки, содержащие списки (например, `['James Cameron']`), в Python-списки:
- Стандартизировать язык (`original_language`) и страну производства (`production_countries`).


### **4. Финансовые поля**
- Преобразовать значения в `budget`, `revenue`, `profit`, `roi` в миллионы долларов:


### **5. Проверка дубликатов**
- Найти дубликаты по `title` и `imdb_id`

- - -

In [594]:
#  очистка всех переменных (не трогает ядро, не сбрасывает вывод)
from IPython import get_ipython
get_ipython().run_line_magic('reset', '-f')



In [595]:
import pandas as pd
import numpy as np
import ast
df_merged = pd.read_csv('data/raw/sci_fi_movies_merged.csv')

df_cleaned = df_merged.copy()  # создаем копию данных, чтобы не изменять оригинал


/var/folders/j1/p2qvyplx67lfm2kfmhw_tb700000gn/T/ipykernel_8417/665304916.py:4: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_merged = pd.read_csv('data/raw/sci_fi_movies_merged.csv')


#### 1. работа с пропущенными значениями

In [596]:
df_cleaned.dropna(subset=['release_date', 'overview', 'imdb_average'])

,Unnamed: 0,id,title,original_title,overview,release_date,popularity,imdb_average,imdb_count,imdb_id,...,top_cast,secondary_cast,episodic_cast,Awards,Rotten_Tomatoes,Metacritic,Metascore,wins,nominations,awards_type
0,0,2756,The Abyss,The Abyss,A civilian oil rig crew is recruited to conduc...,1989-08-09,3.67,7.40,3082,tt0096754,...,"['Ed Harris', 'Mary Elizabeth Mastrantonio', '...","['John Bedford Lloyd', 'Kimberly Scott', 'Chri...","['J. Kenneth Campbell', 'Peter Ratray', 'Micha...",Won 1 Oscar. 9 wins & 16 nominations total,89%,89.00,NaN,1,0,Oscar
1,1,185,A Clockwork Orange,A Clockwork Orange,"In a near-future Britain, young Alexander DeLa...",1971-12-19,4.28,8.20,13078,tt0066921,...,"['Malcolm McDowell', 'Patrick Magee', 'Carl Du...","['James Marcus', 'Michael Tarn', 'Miriam Karli...","['Michael Gover', 'Godfrey Quigley', 'Madge Ry...",Nominated for 4 Oscars. 12 wins & 26 nominatio...,86%,86.00,NaN,0,4,Oscar
2,2,720321,Breathe,Breathe,"Air-supply is scarce in the near future, forci...",2024-04-04,3.57,5.81,196,tt11540468,...,"['Jennifer Hudson', 'Milla Jovovich', 'Quvenzh...","['Raúl Castillo', 'James Saito', 'Dan Martin',...",[],NaN,14%,14.00,NaN,0,0,No data
3,3,871,Planet of the Apes,Planet of the Apes,Astronaut Taylor crash lands on a distant plan...,1968-02-07,4.09,7.60,3630,tt0063442,...,"['Charlton Heston', 'Roddy McDowall', 'Kim Hun...","['James Daly', 'Linda Harrison', 'Robert Gunne...","['Martin Abrahams', 'Army Archerd', 'James Bac...",Nominated for 2 Oscars. 6 wins & 5 nominations...,86%,86.00,NaN,0,2,Oscar
4,4,760873,The Colony,Tides,In the not-too-distant future: after a global ...,2021-08-26,3.65,5.70,290,tt6506264,...,"['Nora Arnezeder', 'Iain Glen', 'Sarah-Sofie B...","['Joel Basman', 'Kotti Yun', 'Bella Bading', '...",[],10 wins & 4 nominations,54%,54.00,NaN,0,0,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485852,485852,633225,Time of Sheep,A Era das Ovelhas,Times are changing. A mysterious red dust has ...,2018-10-19,0.38,0.00,0,tt8779764,...,['Jorge Mota'],[],[],NaN,NaN,NaN,NaN,0,0,No data
485853,485853,615076,Exposure 36,Exposure 36,"A dejected young photographer, turned low leve...",2021-10-20,0.38,5.00,6,tt10518732,...,"['Charles Ouda', 'Jennifer Leigh Whitehead', '...","['James David Rich', 'Montgomery Mauro', 'Chuc...",[],1 win,86%,86.00,NaN,0,0,Other
485854,485854,604279,The Kids from 62-F,The Kids from 62-F,A group of kids find classified information fr...,2016-02-24,0.38,4.30,3,tt4172746,...,"['Gigi Cesarè', 'Eden McCoy', 'Will Spencer', ...","['April Marshall-Miller', 'Darius Marcell', 'R...",[],4 wins & 2 nominations,NaN,NaN,NaN,0,0,Other
485855,485855,541526,Messengers,Messengers,Dr. Sarah Chapel returns to the small town of ...,2004-06-27,0.38,5.30,3,tt0347534,...,"['Michele Hicks', 'Erik Jensen', 'Frankie Fais...","['Peter McRobbie', 'Ronald Guttman', 'John Car...","['Elain R. Graham', 'Dan MacDonald', 'Wade Myl...",2 wins,NaN,NaN,NaN,0,0,Other


#### 2. приведение форматов к единому виду

приводим дату релиза к datetime, извлекаем год и фильтруем по дате. также обработаем числовые рейтинги.


In [597]:
# дата релиза
df_cleaned['release_date'] = pd.to_datetime(df_cleaned['release_date'], errors='coerce')

# извлекаем год релиза
df_cleaned['release_year'] = df_cleaned['release_date'].dt.year

# фильтрация по дате — только до 2025 года
df_cleaned = df_cleaned[df_cleaned['release_date'] <= '2025-01-01']

# очищаем проценты и 'N/A', преобразуем в числовой формат
df_cleaned['Rotten_Tomatoes'] = df_cleaned['Rotten_Tomatoes'].str.replace('%', '', regex=False)
df_cleaned['Rotten_Tomatoes'] = pd.to_numeric(df_cleaned['Rotten_Tomatoes'], errors='coerce')

df_cleaned['Metacritic'] = pd.to_numeric(df_cleaned['Metacritic'], errors='coerce')
df_cleaned['Metascore'] = pd.to_numeric(df_cleaned['Metascore'], errors='coerce')

#### 3. очистка текстовых полей

удаляем лишние пробелы в строках, безопасно парсим поля со списками (если они сохранены в строковом виде)  
и стандартизируем язык и страны в нижний регистр. при этом названия фильмов и оригинальные названия приводим к нормальному виду.


In [598]:
import ast
import re

# очищаем пробелы в строках, не меняя регистр
text_columns = ['title', 'original_title', 'overview']
for col in text_columns:
    df_cleaned[col] = df_cleaned[col].apply(lambda x: x.strip() if isinstance(x, str) else x)

# функция безопасного преобразования строки в список
def safe_parse_list(text):
    if isinstance(text, list):
        return text
    if isinstance(text, str) and text.startswith('['):
        try:
            return ast.literal_eval(text)
        except:
            # если кавычек нет, но элементы разделены запятыми
            return [item.strip() for item in re.sub(r'[\[\]]', '', text).split(',') if item.strip()]
    return []

# обрабатываем поля со списками
list_columns = [
    'director', 'writer', 'producer', 'genres', 'production_countries',
    'full_cast', 'top_cast', 'secondary_cast', 'episodic_cast'
]

for col in list_columns:
    df_cleaned[col] = df_cleaned[col].apply(safe_parse_list)


#### 4. финансовые поля

преобразуем денежные столбцы в миллионы долларов


In [599]:
# пересчитываем прибыль до перевода в миллионы долларов
df_cleaned['profit'] = df_cleaned['revenue'] - df_cleaned['budget']

# делим бюджет, выручку и прибыль на миллион для удобства восприятия
money_columns = ['budget', 'revenue', 'profit']
df_cleaned[money_columns] = df_cleaned[money_columns] / 1e6

pd.options.display.float_format = '{:,.2f}'.format




#### 5. проверка дубликатов

ищем дубликаты по названию и imdb_id


In [600]:
# Находим дубли по столбцам 'title' и 'imdb_id'
duplicates = df_cleaned[df_cleaned.duplicated(subset=['title', 'imdb_id'], keep='first')]

# Удаляем дубли
df_cleaned = df_cleaned.drop_duplicates(subset=['title', 'imdb_id'], keep='first')

In [601]:
df_cleaned.info()  #проводим осмотр таблицы

<class 'pandas.core.frame.DataFrame'>
Index: 9369 entries, 0 to 485856
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Unnamed: 0            9369 non-null   int64         
 1   id                    9369 non-null   int64         
 2   title                 9369 non-null   object        
 3   original_title        9369 non-null   object        
 4   overview              9196 non-null   object        
 5   release_date          9369 non-null   datetime64[ns]
 6   popularity            9369 non-null   float64       
 7   imdb_average          9369 non-null   float64       
 8   imdb_count            9369 non-null   int64         
 9   imdb_id               8809 non-null   object        
 10  director              9369 non-null   object        
 11  producer              9369 non-null   object        
 12  writer                9369 non-null   object        
 13  editor               

In [602]:
df_cleaned.head()

,Unnamed: 0,id,title,original_title,overview,release_date,popularity,imdb_average,imdb_count,imdb_id,...,episodic_cast,Awards,Rotten_Tomatoes,Metacritic,Metascore,wins,nominations,awards_type,release_year,profit
0,0,2756,The Abyss,The Abyss,A civilian oil rig crew is recruited to conduc...,1989-08-09,3.67,7.40,3082,tt0096754,...,"[J. Kenneth Campbell, Peter Ratray, Michael Be...",Won 1 Oscar. 9 wins & 16 nominations total,89.00,89.00,NaN,1,0,Oscar,"1,989.00",20.00
1,1,185,A Clockwork Orange,A Clockwork Orange,"In a near-future Britain, young Alexander DeLa...",1971-12-19,4.28,8.20,13078,tt0066921,...,"[Michael Gover, Godfrey Quigley, Madge Ryan, A...",Nominated for 4 Oscars. 12 wins & 26 nominatio...,86.00,86.00,NaN,0,4,Oscar,"1,971.00",24.83
2,2,720321,Breathe,Breathe,"Air-supply is scarce in the near future, forci...",2024-04-04,3.57,5.81,196,tt11540468,...,[],NaN,14.00,14.00,NaN,0,0,No data,"2,024.00",0.00
3,3,871,Planet of the Apes,Planet of the Apes,Astronaut Taylor crash lands on a distant plan...,1968-02-07,4.09,7.60,3630,tt0063442,...,"[Martin Abrahams, Army Archerd, James Bacon, E...",Nominated for 2 Oscars. 6 wins & 5 nominations...,86.00,86.00,NaN,0,2,Oscar,"1,968.00",26.79
4,4,760873,The Colony,Tides,In the not-too-distant future: after a global ...,2021-08-26,3.65,5.70,290,tt6506264,...,[],10 wins & 4 nominations,54.00,54.00,NaN,0,0,Other,"2,021.00",0.33


In [608]:
## обнаруживаем, что образовалась лишняя колонка Unnamed, а Metacritic и Metascore не несут никакой информации
# удаяем ненужные колонки
df_cleaned = df_cleaned.drop(columns=['Metacritic', 'Metascore', 'Unnamed: 0'])

In [609]:
df_cleaned.to_csv('data/clean/sci_fi_movies_cleaned.csv')

---

## **Итог**

Обновлённый датасет полностью готов к EDA:
- Все ключевые поля имеют корректные типы
- Критические пропуски устранены
- Псевдосписки заменены на настоящие массивы
- Добавлен признак года релиза